<a href="https://colab.research.google.com/github/LeeSeungwon89/Lecture-and-self-study/blob/master/6-2%20k-%ED%8F%89%EA%B7%A0(%EC%A0%95%EB%A6%AC%20%EC%A4%91).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# k-평균

- 이전 절에서는 어떤 과일 사진인지 아는 상태로 각 과일 이미지의 평균값을 구하여 가장 가까운 사진을 골랐으나, 실제 비지도 학습에서는 사진에 어떤 과일이 들어 있는지 알 수 없으므로 'k-평균(k-means)' 군집 알고리즘을 사용하여 평균값을 찾음. 이 평균값이 클러스터의 중심에 위치하므로 '클러스터 중심(Cluster Center)' 또는 '센트로이드(Centroid)' 라고 부름.

- k-평균 알고리즘의 작동 방식

 1. 무작위로 k개의 클러스터 중심을 정함.

 1. 각 샘플에서 가장 가까운 클러스터 중심을 찾아서 해당 클러스터의 샘플로 지정함.

 1. 클러스터에 속한 샘플의 평균값으로 클러스터 중심을 변경함.

 1. 클러스터 중심에 변화가 없을 때까지 위 2번으로 돌아가 반복함.

 이를 그림(출처: 혼자 공부하는 머신러닝+딥러닝)과 설명으로 구체화 하면 아래와 같음. 그림에 매겨진 번호는 아래 설명의 목록 번호와 대응함. 과일 위치는 알고리즘을 종료할 때까지 고정된 상태임.

<img src='https://drive.google.com/uc?export=view&id=1GVCIPrek_1XUNZWp8sQCiHPYD5qafdcg'>

1. 먼저 클러스터 중심(빨간 점) 3개를 랜덤하게 지정하고, 클러스터 중심에서 가장 가까운 샘플을 하나의 클러스터로 묶음. 클러스터에는 순서나 번호가 무의미함.

1. 클러스터 중심을 계산하여 이동시킴. 왼쪽 위의 클러스터는 바나나 쪽으로(오른쪽 위로) 중심이 더 이동하고, 맨 아래 클러스터는 사과 쪽으로(왼쪽 위로) 중심이 더 이동하고, 오른쪽 위의 클러스터는 파인애플 쪽으로(아래로) 중심이 더 이동함. 가장 가까운 샘플을 다시 클러스터로 묶으면 클러스터 3개에 바나나, 사과, 파인애플이 3개씩 묶임.

1. 다시 클러스터 중심을 계산하여 클러스터 중심 3개를 클러스터의 가운데 부분으로 이동시킴. 이동된 클러스터 중심에서 가장 가까운 샘플을 클러스터로 묶음. 중심에서 가장 가까운 샘플은 이전 클러스터(2)와 동일하게 구성된 상태임.

1. 만들어진 클러스터에 변동이 없으므로 k-평균 알고리즘을 종료함.

## KMeans 클래스

- n_clusters: 클러스터 개수를 지정하는 매개변수. 기본값은 8.

- n_init: 처음에 랜덤하게 센트로이드를 초기화하기 때문에 여러 번 반복하여 '이너셔(Inertia)'(이너셔의 의미는 아래에서 설명함) 를 기준으로 가장 좋은 결과를 선택함. 이 반복 횟수를 지정하는 매개변수. 기본값은 10.

- max_iter: k-평균 알고리즘의 한 번 실행에서 최적의 센트로이드를 찾기 위해 반복할 수 있는 최대 횟수. 기본값은 200.

### 데이터 준비

In [22]:
!wget https://bit.ly/fruits_300_data -O fruits_300.npy

--2021-07-19 06:11:17--  https://bit.ly/fruits_300_data
Resolving bit.ly (bit.ly)... 67.199.248.10, 67.199.248.11
Connecting to bit.ly (bit.ly)|67.199.248.10|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/rickiepark/hg-mldl/raw/master/fruits_300.npy [following]
--2021-07-19 06:11:17--  https://github.com/rickiepark/hg-mldl/raw/master/fruits_300.npy
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rickiepark/hg-mldl/master/fruits_300.npy [following]
--2021-07-19 06:11:17--  https://raw.githubusercontent.com/rickiepark/hg-mldl/master/fruits_300.npy
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... conne

In [23]:
# k-평균 모델을 훈련하고자 3차원 배열(샘플 개수, 너비, 높이)을 2차원 배열(샘플 개수, 너비 * 높이)로 변경함.
import numpy as np

fruits = np.load('fruits_300.npy')
fruits_2d = fruits.reshape(-1, 100*100)

### 모델 훈련 후 데이터 살피기

In [24]:
# 'KMeans' 클래스를 불러서 모델을 훈련함. 
from sklearn.cluster import KMeans

# 'n_clusters' 매개변수에 클러스터 개수를 3으로 지정함.
km = KMeans(n_clusters = 3, random_state = 42)

km.fit(fruits_2d) # 비지도 학습이므로 타깃 데이터를 사용하지 않음.

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=42, tol=0.0001, verbose=0)

In [25]:
# 'KMeans' 클래스 객체의 'lables_' 속성에 군집된 결과가 저장됨.
# 이 'labels_' 속성에 저장된 배열은 각 샘플이 어떤 레이블에 해당되는지 나타내며
# 배열 길이는 샘플 개수와 같음. 샘플 개수는 300개임.
# 'n_clusters = 3' 으로 지정해서 군집 개수가 3개이기 때문에 배열 값은 0, 1, 2 중 하나임.
print(km.labels_)

[0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 2 0 2 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 2 2 0 0 0 0 0 0 0 0 2 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1]


In [26]:
# 실제 레이블 0, 1, 2가 어떤 과일 이미지를 주로 모았는지 확인하려면 직접 출력하는 편이 좋음.
# 넘파이의 'unique()' 함수를 사용하여 레이블 0, 1, 2로 모은 샘플 개수를 확인함.
# 넘파이 'unique()' 함수는 특정 배열에 있는 원소 종류를 하나씩만 모아서 출력함.
# 이 배열에는 0, 1, 2 밖에 없기 때문에 0, 1, 2를 출력했고,
# 'return_counts' 매개변수를 'True' 로 지정하면 각 원소 개수를 별도 배열로 함께 출력함.
print(np.unique(km.labels_, return_counts = True))
# 첫 번째 클러스터(레이블 0)가 샘플 91개를 모았고,
# 두 번째 클러스터(레이블 1)가 샘플 98개를 모았고,
# 세 번째 클러스터(레이블 2)가 샘플 111개를 모았음.

(array([0, 1, 2], dtype=int32), array([ 91,  98, 111]))


### 넘파이 올림, 반올림, 내림, 버림 함수

In [27]:
# 설명을 이어가기 전에 넘파이 올림, 반올림, 내림, 버림 함수를 설명함.

# np.ceil(수): 올림.
print(np.ceil(3.4), np.ceil(-3.4))
print(np.ceil(3.6), np.ceil(-3.6))
print()

# np.round(수, 자릿수): 반올림.
print(np.round(34.754, 2), np.round(34.755, 2))
print(np.round(34.754, 0), np.round(34.754, -1))
print()

# np.floor(수): 내림.
print(np.floor(3.4), np.floor(-3.4))
print(np.floor(3.6), np.floor(-3.6))
print()

# np.trunc(수): 소숫점을 버림.
print(np.trunc(3.4), np.trunc(-3.4))

4.0 -3.0
4.0 -3.0

34.75 34.76
35.0 30.0

3.0 -4.0
3.0 -4.0

3.0 -3.0


### 유틸리티 함수 만들기

- 유틸리티 함수를 만들기 위해 'def' 문을 사용함. 'def' 는 'define' 의 '정의하다' 에서 앞 글자를 딴 것임.

- 함수를 해석하는 것도 중요하지만 더 나아가 함수를 직접 짜서 활용하는 역량을 갖추는 것이 핵심임. 이를 도모하려면 일단 수많은 함수를 해석하면서 경험을 쌓되, 단 하나의 코드도 허투루 생략하고 넘어가는 일이 없어야 함.

- 해석을 마쳤으면 다시 따라치면서 손에 익히고, 때에 따라선 암기하는 것도 방편 중 하나임. '모방은 창조의 어머니' 라는 말이 있듯이, 반복하여 모방하는 과정을 거치다보면 필시 창조력을 제고할 수 있음.

In [28]:
# 유틸리티 함수 'draw_fruits()' 를 만들어서 각 클러스터가 나타내는 이미지를 출력함.
import matplotlib.pyplot as plt

# 3차원 배열(샘플 개수, 너비, 높이)을 입력받아 가로로 10개씩 이미지를 출력함.
# 샘플 개수에 따라 행, 열 개수를 계산하고 'figsize' 를 지정함.
# 'figsize' 는 'ratio' 매개변수에 비례하여 커짐.
# 'ratio' 매개변수는 1로 고정하며, 숫자를 변경하면 출력되는 이미지 사이즈를 조절할 수 있음.
def draw_fruits(arr, ratio = 1) :

    # 'len()' 함수를 사용하여 원소(샘플) 개수를 'n' 객체에 넣음.
    n = len(arr)

    # 행은 'rows' 객체로 설정함.
    # 샘플 개수 'n' 을 10으로 나눈 값을 'np.ceil()' 함수를 사용하여 올리고 'rows' 객체에 넣음.
    # 한 행에 이미지를 10개씩 넣어야 하므로 사과 샘플 91개를 시각화 한다면 행 9개(사과 90개) + 행 1개(사과 1개)가 필요함.
    # 즉 행 10개가 필요하므로 수를 올려야 함.
    rows = int(np.ceil(n / 10))

    # 열은 'cols' 객체로 설정함.
    # 행(rows)이 1개이면 열 개수(cols)는 샘플 개수(n)이고, 그렇지 않으면 열 개수(cols)는 10개임.
    # 아래에서 다시 설명함.
    cols = n if rows < 2 else 10

    fig, axs = plt.subplots(rows, cols, figsize = (cols * ratio, rows * ratio), 
                            squeeze = False)

    # 2중 'for' 반복문을 사용하여 먼저 첫 행을 따라 이미지를 그리고, 
    # 두 번째 행을 따라 이미지를 그리는 식으로 구성함.
    for i in range(rows) :
        for j in range(cols) :
            if i * 10 + j < n :
                axs[i, j].imshow(arr[i * 10 + j], cmap = 'gray_r')
            axs[i, j].axis('off')

    plt.show()

In [29]:
# 'draw_fruits()' 함수에 'range(5)' 를 대입하여 구체적으로 설명함.
# 사실 2, 3차원 배열을 대입하여야 하나 메커니즘만 설명하는 것이 목적이므로 1차원 배열을 예로 듦.
# 보기 좋도록 전체 주석으로 처리하지 않음.
draw_fruits(range(5)) # '[0, 1, 2, 3, 4]' 로 이루어짐.

    n = len(range(5)) # 샘플 개수는 5개이므로 'n = 5' .

    rows = int(np.ceil(5 / 10)) # 0.5 이므로 올리면 'rows = 1' .

    cols = 5 if 1 < 2 else 10 # 행이 1개이므로 'cols = 5' .
    # 만약 'n = 11' 이라서 'rows = 2' 이 되면 'cols = 10' 이 됨. 행 1개에 열 10개만큼 샘플 10개가 위치함.
    # 아래에 추가로 제시함.

    fig, axs = plt.subplots(1, 5, figsize = (5 * 1, 1 * 1), squeez = False) # 행 1개, 열 5개만큼 그림.

    for i in range(1) : # 'range(1)' 은 '[0]' 만 들어있음.
        for j in range(5) : # 'range(5)' 는 '[0, 1, 2, 3, 4]' .
            if 0 * 10 + 0 < 5 : # 조건식이 참이라면 아래의 그리는 코드로 내려감.
                axs[0, 0].imshow(arr[0 * 10 + 0], cmap = 'gray_r') # 여기서 그림.
            axs[0, 0].axis('off') # 여기까지 마치고 다시 두 번째 for문으로 올라가서 다음 숫자로 명령을 모두 수행하고 맨 위 for문으로 올라감.

    plt.show()

IndentationError: ignored

In [ ]:
# 위 설명에 이어서 'draw_fruits()' 함수에 'range(11)' 를 대입하여 구체적으로 설명함.
draw_fruits(range(11)) # '[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]' 로 이루어짐.

    n = len(range(11)) # 샘플 개수는 11개이므로 'n = 11' .

    rows = int(np.ceil(11 / 10)) # 1.1 이므로 올리면 'rows = 2' .

    cols = 11 if 2 < 2 else 10 # 행이 2개이므로 'cols = 10' .

    fig, axs = plt.subplots(2, 10, figsize = (10 * 1, 2 * 1), squeez = False) # 행 2개, 열 10개만큼 그림.

    for i in range(2) : # 'range(2)' 은 '[0, 1]'.
        for j in range(10) : # 'range(10)' 는 '[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]'.
            if 0 * 10 + 0 < 11 :
            # 만약 행이 두 번째(i = 1)가 되고 열이 두 번째(j = 1)가 된다면 'if 1 * 10 + 1 < 11' 가 되어 'false' 이며
            # for문을 종료하고 'plt.show()' 로 나옴.
                axs[0, 0].imshow(arr[0 * 10 + 0], cmap = 'gray_r')
            axs[0, 0].axis('off')

    plt.show()

### 불리언인덱싱으로 이미지 출력하기

In [ ]:
# 'km.labels_ == 0' 처럼 쓰면
# 'km_labels_' 배열의 값이 0인 위치는 'True', 아닌 값은 'False' 가 출력됨.
print(km.labels_ == 0)

# 'True' 인 위치의 샘플 배열만 출력함.
print(fruits[km.labels_ == 0])

In [ ]:
# 위의 '불리언인덱싱' 방식을 사용하여 'True' 인 위치의 원소만 추출함.
draw_fruits(fruits[km.labels_ == 0])
# 모든 사과 이미지 91개를 올바르게 출력함.

In [ ]:
draw_fruits(fruits[km.labels_ == 1])
# 모든 바나나 이미지 98개를 올바르게 출력함.

In [ ]:
draw_fruits(fruits[km.label`s_ == 2])
# 레이블이 2인 클러스터는 파인애플 111개에 사과 9개, 바나나 2개가 섞여서 출력됨.
# k-평균 알고리즘이 이 샘플들을 완벽하게 구별하지 못했음.
# 물론 훈련 데이터에 타깃 레이블을 제공하지 않았지만 나름 잘 모은 편임.

## 클러스터 중심

- cluster_centers_: 'KMeans' 클래스가 최종적으로 찾은 클러스터 중심이 저장된 'KMeans' 클래스의 속성.

- transform(): 훈련 데이터 샘플에서 클러스터 중심까지 거리로 변환해 주는 'KMeans' 클래스의 메서드. 'StandardScaler' 클래스처럼 특성값을 변환하는 도구로 사용할 수 있음.

- predict(): 가장 가까운 클러스터 중심을 예측 클래스로 출력하는 'KMeans' 클래스의 메서드.

- n_iter_: 알고리즘이 반복한 횟수가 저장된 'KMeans' 클래스의 속성.

## 최적의 k 찾기

- k-평균 알고리즘이 가진 단점 중 하나는 클러스터 개수를 사전에 지정해야 한다는 점이며, 실전에서는 클러스터가 몇 개 존재하는지 알 수 없음. 사실 군집 알고리즘에서 적절한 k 값을 찾을 완벽한 방법은 없고, 몇 가지 도구가 존재하지만 저마다 장단점이 있음. 그 중에 대표적인 방법은 '엘보우(Elbow)' 이며 '이너셔(Inertia)' 의 변화를 관찰하면서 진행함. 먼저 이너셔에 대한 설명을 제시함.

 - 이너셔(Inertia): 클러스터에 속한 샘플이 얼마나 가깝게 모여 있는지 나타내는 값이며, 클러스터 중심과 클러스터에 속한 샘플 사이의 거리를 제곱하여 합한 값임. 일반적으로 클러스터 개수가 늘어나면 개개의 클러스터 크기는 줄어들기 마련이고 이너셔도 함께 줄어듦.
 
 - 엘보우(Elbow): 클러스터 개수를 늘려가면서 '이너셔(Inertia)' 의 변화를 관찰하여 최적의 클러스터 개수를 찾는 방법임. 클러스터 개수를 증가시키면서 이너셔를 그래프로 그리면 감소하는 속도가 꺾이는 지점이 존재하는데, 이 지점부터는 클러스터 개수를 늘려도 클러스터에 잘 밀집된 정도가 크게 개선되지 않음. 즉 이너셔가 크게 줄어들지 않는 것을 의미하며, 이 지점이 팔꿈치 모양처럼 생겨서 엘보우 방법이라고 함.

이를 그림(출처: 혼자 공부하는 머신러닝+딥러닝)으로 설명하면 아래와 같음.

 <img src='https://drive.google.com/uc?export=view&id=15OlwxxvSlerGR8PLTffgNBD-Mm3LeP3u' width='80%'>

In [ ]:
# KMeans 클래스는 자동으로 이너셔를 계산해서 'inertia_' 속성으로 제공함.
# 클러스터 개수 k를 2 ~ 6까지 바꿔가며 KMeans 클래스를 5번 훈련함.
# 'fit()' 메서드로 모델을 훈련한 후 'inertia' 속성에 저장된 이너셔 값을 'inertia' 리스트에 추가함.
inertia = []

for k in range(2, 7) :
    km = KMeans(n_clusters = k, random_state = 42)
    km.fit(fruits_2d)
    inertia.append(km.inertia_)

plt.plot(range(2, 7), inertia)
plt.xlabel('k')
plt.ylabel('inertia')
plt.show()
# 'k = 3' 지점에서 그래프 기울기가 조금 바뀌었음.
# 이 엘보우 지점보다 클러스터 개수가 많아지면 이너셔의 변화가 줄어들면서 군집 효과도 줄어듦.
# 근데 이 그래프에서는 이런 지점이 명확하게 보이진 않음.